In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
API_KEY = "91e97c7b261216d6e6727272d86892a0-da24a042336ef33336d716c451ea6d36"
ACCOUNT_ID = "101-001-24345029-002"
URL = "https://api-fxpractice.oanda.com/v3/"

In [3]:
session = requests.Session()

In [4]:
session.headers.update({
    "Authorization" : f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

In [5]:
params = dict(
count = 10,
granularity = "H1",
price = "MBA"
)

In [6]:
url = f"{URL}/accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
data = response.json()

In [9]:
instruments_list = data['instruments']

In [10]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [11]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [12]:
instruments_dict = {}
for i in instruments_list:
    key = (i['name'])
    instruments_dict[key] = {k: i[k] for k in key_i}

In [13]:
instruments_dict['USD_JPY']

{'name': 'USD_JPY',
 'type': 'CURRENCY',
 'displayName': 'USD/JPY',
 'pipLocation': -2,
 'displayPrecision': 3,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [14]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [15]:
def fetch_candles(pair_name, count=10, granularity="H1"): 
    url = f"{URL}/instruments/{pair_name}/candles"
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
        return response.status_code, data

def get_candles_df(data):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    if len(data) == 0:
        return pd.DataFrame()
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pk1")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [16]:
code, data = fetch_candles("USD_JPY", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [17]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-01-26 06:00:00+00:00,32496,129.330,129.995,129.199,129.959,129.321,129.986,129.190,129.950,129.338,130.004,129.207,129.968
1,2023-01-26 10:00:00+00:00,33095,129.956,130.363,129.627,129.874,129.947,130.313,129.617,129.864,129.966,130.413,129.637,129.884
2,2023-01-26 14:00:00+00:00,41738,129.878,130.621,129.704,130.340,129.868,130.612,129.695,130.331,129.887,130.630,129.713,130.348
3,2023-01-26 18:00:00+00:00,17293,130.336,130.336,130.052,130.238,130.327,130.327,130.044,130.226,130.345,130.345,130.060,130.249
4,2023-01-26 22:00:00+00:00,41895,130.221,130.242,129.496,129.692,130.175,130.217,129.486,129.684,130.267,130.283,129.505,129.701
5,2023-01-27 02:00:00+00:00,20531,129.690,130.156,129.612,129.984,129.681,130.148,129.602,129.975,129.698,130.165,129.621,129.992
6,2023-01-27 06:00:00+00:00,25123,129.980,130.269,129.716,129.854,129.972,130.259,129.707,129.845,129.989,130.280,129.726,129.862
7,2023-01-27 10:00:00+00:00,26587,129.852,129.999,129.558,129.738,129.844,129.991,129.550,129.730,129.860,130.008,129.566,129.746
8,2023-01-27 14:00:00+00:00,34124,129.731,130.070,129.642,129.914,129.723,130.062,129.633,129.907,129.739,130.079,129.650,129.922
9,2023-01-27 18:00:00+00:00,12665,129.914,129.952,129.806,129.832,129.905,129.944,129.776,129.802,129.922,129.960,129.822,129.862


In [18]:
create_data_file("USD_JPY", count=10, granularity="H4")

USD_JPY H4 10 candles, 2023-01-26 06:00:00+00:00 2023-01-27 18:00:00+00:00


In [19]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [20]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2022-06-08 00:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_USD H4 4001 candles, 2020-07-06 01:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_GBP H1 4001 candles, 2022-06-08 01:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_GBP H4 4001 candles, 2020-07-06 01:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_JPY H1 4001 candles, 2022-06-08 01:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_JPY H4 4001 candles, 2020-07-03 17:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_CHF H1 4001 candles, 2022-06-08 01:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_CHF H4 4001 candles, 2020-07-06 01:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_NZD H1 4001 candles, 2022-06-08 00:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_NZD H4 4001 candles, 2020-07-06 01:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_CAD H1 4001 candles, 2022-06-08 01:00:00+00:00 2023-01-27 21:00:00+00:00
EUR_CAD H4 4001 candles, 2020-07-06 01:00:00+00:00 2023-01-27 18:00:00+00:00
EUR_AUD H1 4001 candles, 2022-06-08 01:00:00+00:00 2023-01-27 21:00:00+00:00